## New dataset via webscrapping

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import psycopg2
import psycopg2.extras
from collections import OrderedDict

In [ ]:
page = requests.get("http://crimetool.bocsar.nsw.gov.au/bocsar/")
web_content = BeautifulSoup(page.content, 'html.parser')
suburbid = OrderedDict()
suburbs = web_content.find(class_="ribbon-input search-content").find_all("option")[1:]
for suburb in suburbs:
    attrs = suburb.attrs
    if attrs["title"] == "2":
        suburbid[attrs["value"]] = suburb.text

In [ ]:
# 658 suburbs in Sydney, list of names obtained from Wikipedia, filtering out the suburb ids
sydney_suburb_list = []
suburb_url = requests.get("https://en.wikipedia.org/wiki/List_of_Sydney_suburbs")
sydney_suburb = BeautifulSoup(suburb_url.content, 'html.parser')
sydney_suburb = sydney_suburb.find_all("p")
for suburb in sydney_suburb:
    suburbs = suburb.find_all("a")
    for  i in range(len(suburbs)):
        if "title" in suburbs[i].attrs.keys():
            if suburbs[i]["title"].endswith(", New South Wales"):
                name = suburbs[i]["title"].split(",")[0]
                # cleaning up the data for name differences 
                if name == u'Bankstown Airport':
                    name = u'Bankstown Aerodrome'
                sydney_suburb_list.append(name)

In [ ]:
# some suburbs are in the wiki list but not the suburbid, some suburbs have the same name, need to get rid of that
# 654 + 16 (duplicates) = 670
sydney = OrderedDict()
for i in range(len(sydney_suburb_list)):
    for k, v in suburbid.items():
        v = v.replace("(Suburb)","")
        if v.startswith(sydney_suburb_list[i].upper()):
            sydney[k] = v

In [ ]:
# getting offenceids
offenceid = OrderedDict()
offences = web_content.find(class_="offence_list").find("nav:group")
while offences != None:
    attrs = offences.attrs
    if attrs["datavalue"] == "" and attrs["title"] == "More Offences":
        other = offences.find("nav:item")
        while other!= None:
            offenceid[other["datavalue"]] = other["title"]
            other = other.find_next("nav:item")
    if attrs["datavalue"] != "":
        offenceid["-"+attrs["datavalue"]] = attrs["title"]
    offences = offences.find_next("nav:group")

In [ ]:
import geojson
import pprint
import random
import time 
base_url =  "http://crimetool.bocsar.nsw.gov.au/bocsar/api/"
stats = "TableDataQuick"
regionids = [x for x in sydney.keys()]
crimelist= []
# url length limit is 500, otherwise would throw a 414 error, we split the query two parts
def apiCall(start, finish):
    flag = 0
    for offence in offenceid.keys():
        statparams = {"RegionId":regionids[start:finish],"RegionTypeId":2,"Start_Year":2017,"End_Year":2018,"Month":12,"DataType":"I",\
    "OffenceId":offence}
        response = requests.get(base_url + stats, params = statparams)
        results = response.json()
        data = results["data"]
        if flag == 0:
            for i in range(len(data)):
                flag = 1
                crime = {"2018_count":data[i]["2018_count"],"2018_pop":data[i]["2018_pop"],\
                            "RegionId":data[i]["REGIONID"], "name":data[i]["NAME"]}
                crimelist.append(crime)
        elif flag == 1:
            for i in range(len(data)):
                crimelist[start+i]["2018_count"] += data[i]["2018_count"]
        sleeptime = random.randint(1,5)
        time.sleep(sleeptime)
apiCall(0,500)
apiCall(500,len(regionids))

In [ ]:
# now to append the geometry data and store in geojson 
# note that in the API call, results["data"][0] is for the whole of New South Wales
# as we split and repeated the work twice, need to delete entry 501
del crimelist[501]
crimefeatures = []
for i in range(len(regionids)):
    insert = crimelist[i+1]
    url = base_url + "Geometry/" + regionids[i]
    geo = requests.get(url)
    geoda]ta = geo.json()["geometry"]
    feature = geojson.Feature(properties={"2018_count":insert["2018_count"], \
                                                                    "2018_pop":insert["2018_pop"], \
                                                                    "RegionId":insert["RegionId"], \
                                                                    "name":insert["name"] \
                                                                    })
    feature["geometry"] = geodata
    crimefeatures.append(feature)
    time.sleep(random.randint(1,5))
crime_collection = geojson.FeatureCollection(crimefeatures) 
with open('crimefeatures.geojson', 'w') as f:
   geojson.dump(crime_collection, f)

## Import SA2 shapefile

In [ ]:
import shapefile
sf = shapefile.Reader("SA2/SA2_2016_AUST.shp", encoding="iso-8859-1")

In [ ]:
import psycopg2
import psycopg2.extras

def pgconnect():
    try: 
        conn = psycopg2.connect(host='',
                                database='',
                                user='',
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

conn = pgconnect()

In [ ]:
def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None and msg:
                    print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

In [ ]:
aust_schema = '''CREATE TABLE AUST (
                     SA2_MAIN16 INTEGER, 
                     SA2_5DIG16 INTEGER, 
                     SA2_NAME16 VARCHAR(80), 
                     SA3_CODE16 INTEGER,
                     SA3_NAME16 VARCHAR(80),
                     SA4_CODE16 INTEGER,
                     SA4_NAME16 VARCHAR(80),
                     GCC_CODE16 VARCHAR(80),
                     GCC_NAME16 VARCHAR(80),
                     STE_CODE16 INTEGER,
                     STE_NAME16 VARCHAR(80),
                     AREASQKM16 INTEGER,
                     geom GEOMETRY(Polygon,4326),
                     CONSTRAINT aust_area_id_fkey FOREIGN KEY (SA2_MAIN16)
                     REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)''' 

pgquery(conn, "DROP TABLE AUST",msg="cleared old table")
pgquery(conn, aust_schema, msg="created AUST table")

In [ ]:
import re

insert_stmt = """INSERT INTO AUST VALUES ( %(SA2_MAIN16)s, %(SA2_5DIG16)s, %(SA2_NAME16)s, %(SA3_CODE16)s, %(SA3_NAME16)s,
                                            %(SA4_CODE16)s, %(SA4_NAME16)s,%(GCC_CODE16)s,%(GCC_NAME16)s,%(STE_CODE16)s,
                                            %(STE_NAME16)s,%(AREASQKM16)s,
                                            ST_GEOMFROMTEXT(%(geom)s, 4326) )"""

shapes = sf.shapes()
records= sf.records()

row = {}
for i in range(0, len(shapes)):
    record = sf.record(i)
    shape  = sf.shape(i)
    row['SA2_MAIN16']=record[0]
    row['SA2_5DIG16']=record[1]
    row['SA2_NAME16']=record[2]
    row['SA3_CODE16']=record[3]
    row['SA3_NAME16']=record[4]
    row['SA4_CODE16']=record[5]
    row['SA4_NAME16']=record[6]
    row['GCC_CODE16']=record[7]
    row['GCC_NAME16']=record[8]
    row['STE_CODE16']=record[9]
    row['STE_NAME16']=record[10]
    row['AREASQKM16']=record[11]    
    row['geom']="POLYGON(("
    i=0
    for x, y in shape.points:
       row['geom']+="%s %s," % (x,y)
       # check for start of a new polygon part
       i += 1
       if i in shape.parts:
           row['geom']= re.sub(",$", "),(", row['geom'])
    # properly end the polygon string
    row['geom'] = re.sub(",$", "))", row['geom'])
    # finally: insert new row into the table
    if row['GCC_NAME16']=='Greater Sydney':pgquery(conn, insert_stmt, args=row, msg="inserted "+str(record[2]))

## Import csv and scrapped data into database

In [ ]:
#load five files
import csv
import pandas as pd
data_bikesharingpods=list(csv.DictReader(open('data/BikeSharingPods.csv')))
data_businessstats=list(csv.DictReader(open('data/BusinessStats.csv')))
data_censusstats=list(csv.DictReader(open('data/CensusStats.csv')))
data_neighbourhoods=list(csv.DictReader(open('data/Neighbourhoods.csv')))
data_statisticalareas=list(csv.DictReader(open('data/StatisticalAreas.csv')))

In [ ]:
import pandas as pd

data_businessstats1=pd.read_csv('data/BusinessStats.csv')
business_mean={}
for i in data_businessstats1:
    business_mean[i]=int(data_businessstats1[i].mean()+0.5)
    
data_censusstats1=pd.read_csv('data/CensusStats.csv')
censuss_mean={}
for i in data_censusstats1:
    censuss_mean[i]=int(data_censusstats1[i].mean()+0.5)

data_neighbourhoods1=pd.read_csv('data//Neighbourhoods.csv')
neighbourhoods_mean={}
for i in data_neighbourhoods1:
    if i!='area_name':neighbourhoods_mean[i]=int(data_neighbourhoods1[i].mean()+0.5)

In [ ]:
statisticalareas_schema=''' CREATE TABLE IF NOT EXISTS statisticalareas(
                        area_id integer NOT NULL,
                        area_name varchar(80),
                        parent_area_id integer,
                        CONSTRAINT statisticalareas_pkey PRIMARY KEY (area_id))'''


pgquery(conn, "DROP TABLE statisticalareas",msg="cleared old table")
pgquery(conn, statisticalareas_schema, msg="created statisticalareastable")

In [ ]:
insert_stmt = """INSERT INTO statisticalareas(area_id,area_name,parent_area_id) VALUES (%(area_id)s, %(area_name)s, %(parent_area_id)s)"""
for row in data_statisticalareas:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
businessstats_schema=''' CREATE TABLE IF NOT EXISTS businessstats(
                            area_id integer NOT NULL,
                            num_businesses integer,
                            retail_trade integer,
                            accommodation_and_food_services integer,
                            health_care_and_social_assistance integer,
                            education_and_training integer,
                            arts_and_recreation_services integer,
                            CONSTRAINT bussinessstats_pkey PRIMARY KEY (area_id),
                            CONSTRAINT bussinessstats_area_id_fkey FOREIGN KEY (area_id)
                            REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''

pgquery(conn, "DROP TABLE businessstats",msg="cleared old table")
pgquery(conn, businessstats_schema, msg="created businessstats")

In [ ]:
insert_stmt = """INSERT INTO businessstats(area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,
education_and_training,arts_and_recreation_services) VALUES (%(area_id)s, %(num_businesses)s, %(retail_trade)s,
%(accommodation_and_food_services)s,%(health_care_and_social_assistance)s, %(education_and_training)s, 
%(arts_and_recreation_services)s)"""
for row in data_businessstats:
    #use average value to handle null
    if row['num_businesses']=='':row['num_businesses']=business_mean['num_businesses']
    if row['retail_trade']=='':row['retail_trade']=business_mean['retail_trade']
    if row['accommodation_and_food_services']=='':row['accommodation_and_food_services']=business_mean['accommodation_and_food_services']
    if row['health_care_and_social_assistance']=='':row['health_care_and_social_assistance']=business_mean['health_care_and_social_assistance']
    if row['education_and_training'] == '': row['education_and_training']=business_mean['education_and_training']
    if row['arts_and_recreation_services']=='':row['arts_and_recreation_services']=business_mean['arts_and_recreation_services']
for row in data_businessstats:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
censusstats_schema=''' CREATE TABLE IF NOT EXISTS censusstats(
                                area_id integer NOT NULL,
                                median_annual_household_income integer,
                                avg_monthly_rent integer,
                                CONSTRAINT censusstats_pkey PRIMARY KEY (area_id),
                                CONSTRAINT censusstats_area_id_fkey FOREIGN KEY (area_id)
                                REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''

pgquery(conn, "DROP TABLE censusstats",msg="cleared old table")
pgquery(conn, censusstats_schema, msg="created censusstats")

In [ ]:
insert_stmt = """INSERT INTO censusstats(area_id,median_annual_household_income,avg_monthly_rent) VALUES (%(area_id)s, %(median_annual_household_income)s, %(avg_monthly_rent)s)"""
for row in data_censusstats:
    if row['median_annual_household_income']=='':row['median_annual_household_income']=censuss_mean['median_annual_household_income']
    if row['avg_monthly_rent']=='':row['avg_monthly_rent']=censuss_mean['avg_monthly_rent']
for row in data_censusstats:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
neighbourhoods_schema=''' CREATE TABLE IF NOT EXISTS neighbourhoods(
                                area_id integer NOT NULL,
                                area_name character varying(80),
                                land_area double precision,
                                population integer,
                                number_of_dwellings integer,
                                number_of_businesses integer,
                                CONSTRAINT neighbourhoods_pkey PRIMARY KEY (area_id),
                                CONSTRAINT neighbourhoods_area_id_fkey FOREIGN KEY (area_id)
                                REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''

pgquery(conn, "DROP TABLE neighbourhoods",msg="cleared old table")
pgquery(conn, neighbourhoods_schema, msg="created neighbourhoods")

In [ ]:
insert_stmt = """INSERT INTO neighbourhoods(area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses) 
                VALUES (%(area_id)s, %(area_name)s, %(land_area)s,%(population)s,%(number_of_dwellings)s,%(number_of_businesses)s)"""
for row in data_neighbourhoods:
    for i in row:
        if row[i]=='':row[i]=neighbourhoods_mean[i]
for row in data_neighbourhoods:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
bikesharingpods_schema='''CREATE TABLE public.bikesharingpods(
                          station_id integer NOT NULL,
                          name varchar(80) ,
                          num_bikes integer,
                          num_scooters integer,
                          geom GEOMETRY(Point,4326),
                          description text COLLATE pg_catalog."default",
                          CONSTRAINT bikesharingpods_pkey PRIMARY KEY (station_id))'''
pgquery(conn, "DROP TABLE bikesharingpods",msg="cleared old table")
pgquery(conn, bikesharingpods_schema, msg="created bikesharingpods")

In [ ]:
insert_stmt = """INSERT INTO bikesharingpods(station_id ,name,num_bikes,num_scooters,geom,description) 
                VALUES (%(station_id)s, %(name)s, %(num_bikes)s,%(num_scooters)s,ST_GEOMFROMTEXT(%(geom)s, 4326) ,%(description)s)"""
for row in data_bikesharingpods:
    row['geom']='Point('+row['longitude']+' '+row['latitude']+')'
for row in data_bikesharingpods:
    pgquery(conn, insert_stmt, row, "row inserted")

## Import extra dataset

In [ ]:
import geopandas as gpd
crimefeatures = gpd.read_file('data/crimefeatures.geojson')

In [ ]:
crimefeatures_schema = '''CREATE TABLE CRIMEFEATURES (
                     count INTEGER, 
                     pop INTEGER, 
                     name VARCHAR(80), 
                     RegionId INTEGER,
                     geom GEOMETRY(MULTIPOLYGON,4326),
                     CONSTRAINT crimefeatures_pkey PRIMARY KEY (regionid))'''
pgquery(conn, "DROP TABLE crimefeatures",msg="cleared old table")
pgquery(conn, crimefeatures_schema, msg="created crimefeatures table")

In [ ]:
insert_stmt = """INSERT INTO CRIMEFEATURES(count,pop,name,regionid,geom) VALUES (%(count)s, %(pop)s, %(name)s,%(regionid)s,ST_GEOMFROMTEXT(%(geom)s, 4326))"""
values={}
for row in crimefeatures.values:
    values['count']=row[0]
    values['pop']=row[1]
    values['name']=row[2]
    values['regionid']=row[3]
    values['geom']=str(row[4])
    pgquery(conn, insert_stmt, values, "row inserted")

# Predict the numbers crimes for areas

## Get the training data and the testing data

All areas that have corresponding crime count are included in the trainging set. Also, 4 attributes are selected for the learning, which are the population, the number of businesses, the median annual household income and average monthly rent. 

In [ ]:
# Connect to the database
conn = pgconnect()

# Query the training data
query = '''SELECT N.area_id, N.population, N.number_of_businesses, median_annual_household_income AS income, avg_monthly_rent AS rent, CF.count
             FROM Neighbourhoods N JOIN CensusStats CS USING (area_id)
                                   JOIN CrimeFeatures CF ON (LOWER(N.area_name) = LOWER(CF.name));'''

training_data = pgquery(conn, query)
print(len(training_data))
print(training_data)

# Query the testing data
query = '''SELECT N.area_id, N.population, N.number_of_businesses, median_annual_household_income AS income, avg_monthly_rent AS rent
             FROM Neighbourhoods N JOIN CensusStats USING (area_id)
            WHERE LOWER(N.area_name) NOT IN (SELECT LOWER(name) FROM CrimeFeatures);'''

testing_data = pgquery(conn, query)
print(testing_data)

## Make predictions

A decision tree regression model is built based on the training set. Then the model makes predicitons for each entry in the testing data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Exact attributes and targets for all data
training_attrs = []
training_targets = []

for entry in training_data:
    training_attrs.append(entry[1:-1])
    training_targets.append(entry[-1])

# Build a decision tree regressor
regr = DecisionTreeRegressor()
regr.fit(training_attrs, training_targets)

# Exact attributes of each testing entry
testing_attrs = []

for entry in testing_data:
    testing_attrs.append(entry[1:])
    
# Make predictions
predictions = regr.predict(testing_attrs)

# Put the predictions back to the testing data
result = []
for i in range(len(testing_data)):
    
    entry = []
    
    # TODO: Only the first element is needed.
    for elem in testing_data[i]:
        entry.append(elem)
    
    entry.append(predictions[i]);
    
    result.append(entry)

# Print the result
for entry in result:
    print(entry)

## Put the training data and the testing data together

Now all data are put together in one list, which is ready for the calulation.

In [ ]:
for entry in training_data:
    if entry[0] >= 99999999:
        result.append(entry)
len(result)

## Put the result into postgresql

In [ ]:
predictedresult_schema=''' CREATE TABLE IF NOT EXISTS predictedresult(
                                area_id integer NOT NULL,
                                population integer,
                                crime_count integer,
                                CONSTRAINT predictedresult_pkey PRIMARY KEY (area_id),
                                CONSTRAINT predictedresult_area_id_fkey FOREIGN KEY (area_id)
                                REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)
                                '''

pgquery(conn, "DROP TABLE predictedresult",msg="cleared old table")
pgquery(conn, predictedresult_schema, msg="created predictedresult")

In [ ]:
insert_stmt = """INSERT INTO predictedresult(area_id,population,crime_count) VALUES (%(area_id)s, %(population)s, %(crime_count)s)"""
values={}
for row in result:
    values['area_id']=row[0]
    values['population']=row[1]
    values['crime_count']=row[5]
    pgquery(conn, insert_stmt, values, "row inserted")

## Calculate Cyclability-score as given 

In [ ]:
import numpy as np
podsquery='''select area_id,count(station_id),land_area,count(station_id)/land_area as bikepod_density
from neighbourhoods n left outer join 
(select *
from bikesharingpods b , aust au
where st_contains(au.geom,b.geom))
as pods
on (pods.sa2_main16=n.area_id)
group by area_id
order by area_id'''
podsresponse=pgquery( conn, podsquery)
podsmeasures=[]
for row in podsresponse:
    podsmeasures+=[row[3]]
podsmean=np.mean(podsmeasures)
podsstd=np.std(podsmeasures)
zbikepods=[]
for row in podsresponse:
    newdic={}
    newdic['area_id']=row[0]
    newdic['z_bikepods_density']=(row[3]-podsmean)/podsstd
    zbikepods+=[newdic]

In [ ]:
zbikepods_schema=''' CREATE TABLE IF NOT EXISTS zbikepods(
                        area_id integer NOT NULL,
                        z_bikepods_density float,
                        CONSTRAINT zbikepods_pkey PRIMARY KEY (area_id),
                        CONSTRAINT zbikepods_area_id_fkey FOREIGN KEY (area_id)
                        REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''
pgquery(conn, "DROP TABLE zbikepods",msg="cleared old table")
pgquery(conn, zbikepods_schema, msg="created zbikepods table")

In [ ]:
insert_stmt = """INSERT INTO zbikepods(area_id,z_bikepods_density) VALUES (%(area_id)s, %(z_bikepods_density)s)"""
for row in zbikepods:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
populationquery='''select area_id,(population/land_area-avg)/sd as z_population_density from neighbourhoods N 
,(select avg(population/land_area) as "avg",stddev(population/land_area) as sd from neighbourhoods) as st
'''
populationresponse=pgquery( conn, populationquery)
zpopulation=[]
for row in populationresponse:
    newdic={}
    newdic['area_id']=row[0]
    newdic['z_population_density']=row[1]
    zpopulation+=[newdic]

In [ ]:
zpopulation_schema=''' CREATE TABLE IF NOT EXISTS zpopulation(
                        area_id integer NOT NULL,
                        z_population_density float,
                        CONSTRAINT zpopulation_pkey PRIMARY KEY (area_id),
                        CONSTRAINT zpopulation_area_id_fkey FOREIGN KEY (area_id)
                        REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''
pgquery(conn, "DROP TABLE zpopulation",msg="cleared old table")
pgquery(conn, zpopulation_schema, msg="created zpopulation table")

In [ ]:
insert_stmt = """INSERT INTO zpopulation(area_id,z_population_density) VALUES (%(area_id)s, %(z_population_density)s)"""
for row in zpopulation:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
dwellingquery='''select area_id,(number_of_dwellings/land_area-avg)/sd as z_number_of_dwellings_density 
from neighbourhoods N ,
(select avg(number_of_dwellings/land_area) as "avg",stddev(number_of_dwellings/land_area) as sd 
 from neighbourhoods) as st'''
dwellingresponse=pgquery( conn, dwellingquery)
zdwelling=[]
for row in dwellingresponse:
    newdic={}
    newdic['area_id']=row[0]
    newdic['z_dwelling_density']=row[1]
    zdwelling+=[newdic]

In [ ]:
zdwelling_schema=''' CREATE TABLE IF NOT EXISTS zdwelling(
                        area_id integer NOT NULL,
                        z_dwelling_density float,
                        CONSTRAINT zdwelling_pkey PRIMARY KEY (area_id),
                        CONSTRAINT zdwelling_area_id_fkey FOREIGN KEY (area_id)
                        REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''
pgquery(conn, "DROP TABLE zdwelling",msg="cleared old table")
pgquery(conn, zdwelling_schema, msg="created zdwelling table")

In [ ]:
insert_stmt = """INSERT INTO zdwelling(area_id,z_dwelling_density) VALUES (%(area_id)s, %(z_dwelling_density)s)"""
for row in zdwelling:
    pgquery(conn, insert_stmt, row, "row inserted")

In [ ]:
servicequery='''select area_id,(measure-avg)/stddev
from(
select avg(measure),stddev(measure)
from
(select area_id,num_businesses*a/sum+retail_trade*b/sum+accommodation_and_food_services*c/sum+health_care_and_social_assistance*d/sum+
education_and_training*e/sum+arts_and_recreation_services*f/sum as measure
from businessstats,
(select avg(num_businesses) as a,avg(retail_trade) as b,avg(accommodation_and_food_services
) as c,avg(health_care_and_social_assistance
) as d,avg(education_and_training
) as e,avg(arts_and_recreation_services
) as f,avg(num_businesses)+avg(retail_trade)+avg(accommodation_and_food_services
)+avg(health_care_and_social_assistance
)+avg(education_and_training
)+avg(arts_and_recreation_services
) as sum  from businessstats) as st)
as mea
) as std,
(select area_id,num_businesses*a/sum+retail_trade*b/sum+accommodation_and_food_services*c/sum+health_care_and_social_assistance*d/sum+
education_and_training*e/sum+arts_and_recreation_services*f/sum as measure
from businessstats,
(select avg(num_businesses) as a,avg(retail_trade) as b,avg(accommodation_and_food_services
) as c,avg(health_care_and_social_assistance
) as d,avg(education_and_training
) as e,avg(arts_and_recreation_services
) as f,avg(num_businesses)+avg(retail_trade)+avg(accommodation_and_food_services
)+avg(health_care_and_social_assistance
)+avg(education_and_training
)+avg(arts_and_recreation_services
) as sum  from businessstats) as st)
as mea'''
serviceresponse=pgquery( conn, servicequery)
zservice=[]
for row in serviceresponse:
    newdic={}
    newdic['area_id']=row[0]
    newdic['z_service_balance']=row[1]
    zservice+=[newdic]

In [ ]:
zservice_schema=''' CREATE TABLE IF NOT EXISTS zservice(
                        area_id integer NOT NULL,
                        z_service_balance float,
                        CONSTRAINT zservice_pkey PRIMARY KEY (area_id),
                        CONSTRAINT zservice_area_id_fkey FOREIGN KEY (area_id)
                        REFERENCES public.statisticalareas (area_id) MATCH SIMPLE)'''
pgquery(conn, "DROP TABLE zservice",msg="cleared old table")
pgquery(conn, zservice_schema, msg="created zservice table")

In [ ]:
insert_stmt = """INSERT INTO zservice(area_id,z_service_balance) VALUES (%(area_id)s, %(z_service_balance)s)"""
for row in zservice:
    pgquery(conn, insert_stmt, row, "row inserted")

# Calculate the Cyclability Score and the Correlation

In [ ]:
conn = pgconnect()

query = '''SELECT N.area_id,
                  N.area_name,
                  median_annual_household_income AS income,
                  avg_monthly_rent AS rent,
                  z_bikepods_density + z_population_density + z_dwelling_density + z_service_balance + z_crime AS score
             FROM Neighbourhoods N JOIN CensusStats USING (area_id)
                                   JOIN ZBikePods   USING (area_id)
                                   JOIN ZPopulation USING (area_id)
                                   JOIN ZDwelling   USING (area_id)
                                   JOIN ZService    USING (area_id)
                                   JOIN ZCrime      USING (area_id);'''

res = pgquery(conn, query)

query = '''SELECT MAX(z_bikepods_density + z_population_density + z_dwelling_density + z_service_balance + z_crime),
                  MIN(z_bikepods_density + z_population_density + z_dwelling_density + z_service_balance + z_crime)
             FROM Neighbourhoods N JOIN ZBikePods   USING (area_id)
                                   JOIN ZPopulation USING (area_id)
                                   JOIN ZDwelling   USING (area_id)
                                   JOIN ZService    USING (area_id)
                                   JOIN ZCrime      USING (area_id);'''

max_score, min_score = pgquery(conn, query)[0]

data = []

for row in res:
    
    entry = []
    
    for i in range(4):
        entry.append(row[i])
        
    entry.append((row[-1] - min_score) / (max_score - min_score) * 100)
    
    data.append(entry)

scores = [entry[-1] for entry in data]
income = [entry[2] for entry in data]
rent = [entry[3] for entry in data]

In [ ]:
import matplotlib.pyplot as plt

bp_dict = plt.boxplot(scores, labels=['cyclability score'], vert=False, showmeans=True)

for line in bp_dict['medians']:
    x, y = line.get_xydata()[1]
    plt.text(x, y, '%.1f' % x, horizontalalignment='center')

for line in bp_dict['boxes']:
    x, y = line.get_xydata()[0]
    plt.text(x,y, '%.1f' % x, horizontalalignment='center', verticalalignment='top')
    x, y = line.get_xydata()[3]
    plt.text(x,y, '%.1f' % x, horizontalalignment='center', verticalalignment='top')

    
plt.show()

## Correlation

### Calculation

In [ ]:
corr_coef = np.corrcoef([income, rent, scores])
print(corr_coef)

According the output of the `corrcoef` funciton, the correlation coefficient of income and scores is $0.24127378$ and the correlation coefficient of rent and scores is $0.40395861$.

### Visualisation

In [ ]:
def plot(x, y, x_label=None, y_label=None, title=None):
    
    plt.scatter(x, y, s=7.0, c='#EB522C')

    axes = plt.gca()
    a, b = np.polyfit(x, y, 1)
    x_plot = np.linspace(axes.get_xlim()[0], axes.get_xlim()[1], 100)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.plot(x_plot, a * x_plot + b, '-')

#### Income - Scores

In [ ]:
plot(income, scores, 'median annual household income', 'cyclability score', 'median annual household income - cyclability score')

#### Rent - Scores

In [ ]:
plot(rent, scores, 'average monthly rent', 'cyclability score', 'average monthly rent - cyclability score')

#### Comparision

In [ ]:
plt.matshow(corr_coef)
plt.colorbar()
plt.summer()
plt.xticks(np.arange(3), ('Income', 'Rent', 'Score'))
plt.yticks(np.arange(3), ('Income', 'Rent', 'Score'))
plt.show()

## Visual Mapping